In [200]:
import folium
import geopandas
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import jenkspy

In [130]:
# m = folium.Map(location=[45.5236, -122.6750])
# m

In [131]:
lines = geopandas.read_file("capas/mig_recientes_2011.gpkg", layer='flujos')

In [163]:
# lista con nombres de deptos para ser usadas en etiquetas
names = ['Montevideo','Artigas','Canelones','Cerro Largo','Colonia','Durazno','Flores','Florida','Lavalleja',
         'Maldonado','Paysandú','Río Negro','Rivera','Rocha','Salto','San José','Soriano','Tacuarembó','Treinta y Tres']

In [191]:
# Initialise the Scaler 
scaler = MinMaxScaler(feature_range=(1, 30))
  
# To scale data 
X = np.array(lines.personas_mig)
X_scaled = scaler.fit_transform(X[:, np.newaxis], y=None)

In [192]:
lines['personas_mig_scaled'] = X_scaled

In [193]:
lines.head()

,personas_mig,depto_origen,depto_destino,geometry,personas_mig_scaled
0,914,1,2,"LINESTRING (571906.031 6146848.946, 503900.219...",1.798430
1,74,3,2,"LINESTRING (598065.774 6177339.485, 503900.219...",1.063034
2,15,4,2,"LINESTRING (749014.919 6407567.643, 503900.219...",1.011381
3,28,5,2,"LINESTRING (439070.917 6223291.295, 503900.219...",1.022762
4,32,6,2,"LINESTRING (590741.575 6345718.477, 503900.219...",1.026264


In [204]:
lines['nbreaks'] = pd.cut(
    lines['personas_mig'],
    bins=jenkspy.jenks_breaks(lines['personas_mig'], nb_class=4),
#     labels=['bucket_1', 'bucket_2', 'bucket_3', 'bucket_4'],
    labels=['#deadad', '#f18f8c','#ff493d', '#ff0000'],
    include_lowest=True)

lines

,personas_mig,depto_origen,depto_destino,geometry,personas_mig_scaled,nbreaks
0,914,1,2,"LINESTRING (571906.031 6146848.946, 503900.219...",1.798430,#deadad
1,74,3,2,"LINESTRING (598065.774 6177339.485, 503900.219...",1.063034,#deadad
2,15,4,2,"LINESTRING (749014.919 6407567.643, 503900.219...",1.011381,#deadad
3,28,5,2,"LINESTRING (439070.917 6223291.295, 503900.219...",1.022762,#deadad
4,32,6,2,"LINESTRING (590741.575 6345718.477, 503900.219...",1.026264,#deadad
...,...,...,...,...,...,...
337,2481,15,1,"LINESTRING (498335.337 6534101.727, 571906.031...",3.170294,#f18f8c
338,1852,16,1,"LINESTRING (525216.795 6203635.210, 571906.031...",2.619623,#f18f8c
339,1922,17,1,"LINESTRING (430375.064 6291882.308, 571906.031...",2.680906,#f18f8c
340,2611,18,1,"LINESTRING (615975.221 6446905.919, 571906.031...",3.284106,#f18f8c


In [195]:
# exportar a GeoJSON un archivo por cada departamento
# for i in range(1,20):
#     lines_filtered = lines.loc[lines.depto_origen== i]
#     file_name = "capas/geojson/mig_recientes_2011_depori_{}.geojson".format(i)
#     lines_filtered.to_file(file_name, driver='GeoJSON')

In [196]:
# prueba: enlistar un GeoJSON filtrando sólo Mvdeo.
# lines_json = lines.loc[lines.depto_origen== 1].to_crs('EPSG:4326').to_json()

In [209]:
# enlista un GeoJSON por depto
lines_json = [lines.loc[lines.depto_origen==x].to_crs('EPSG:4326').to_json() for x in range(1,20)]

mapa = folium.Map([-32, -56], zoom_start=7, tiles='cartodb positron')

   
for i in range(0,19):
    if i==0:
        show_val = 1
    else:
        show_val = 0
    folium.features.GeoJson(data=lines_json[i],
                                  tooltip=folium.features.GeoJsonTooltip(
                                        fields=['personas_mig'],
                                        aliases=['Migrantes:'], 
                                        labels=True, 
                                        sticky=False),
                                  name=names[i],
                                  show=show_val,
                                  style_function= lambda x: {
                                      "weight": x['properties']['personas_mig_scaled'],
                                      "color": x['properties']['nbreaks'],
                                  },).add_to(mapa)
    
# style_function = lambda x: {x['properties']['personas_mig']}

# https://python-visualization.github.io/folium/modules.html#folium.features.GeoJson



folium.LayerControl(collapsed=False).add_to(mapa) 

mapa

# mapa.save('prueba_1.html')